In [1]:
import sys, os
from pathlib import Path
os.chdir(Path('..'))
# os.listdir()

In [2]:
import pyarrow as pa
import pyarrow.csv as csv
from zipfile import ZipFile

import numpy as np
import pandas as pd
from datetime import datetime, date
import re

from src.pipeline.etc import last_business_day, next_business_day
import src.server.insert
import src.server.secret
import src.server.query
from src.pipeline.tables import zipfiles
from src.server.query import query

In [3]:
servername  = src.server.secret.servername
database    = src.server.secret.database

today = date.today()
today_str = today.strftime('%Y-%m-%d')
yesterday = last_business_day(today)
d2 = last_business_day(yesterday).strftime('%Y-%m-%d')
tomorrow = next_business_day(today).strftime('%Y-%m-%d')

In [12]:
extract = Path('data/load')
# file = extract / f'{today_str}.zip'
file = extract / f'{tomorrow}.zip'
# t0 = pd.read_csv(file)
# file = extract / f'{yesterday}.zip'
t0 = pd.read_csv(file)
# with ZipFile(file, 'r') as zips:
#         zips.extractall(extract)
#         file = extract / zips.namelist()[0]
#         df0 = csv.read_csv(file).to_pandas()
#         os.remove(file)

C:\Users\ARoethe\Anaconda3\envs\offsite\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8,26,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
test = t0[t0.Last_Call.notna()].copy()
test['buzday'] = test['Last_Call'].apply(lambda x: len(pd.bdate_range(x, today_str)) )
# test[['Last_Call','age','age2']]
# t = test['Last_Call'].apply(lambda x: len(pd.bdate_range(x, today_str)) )

In [8]:
dubs = test.append(t0)
unique = dubs.drop_duplicates(['OutreachID']).reset_index(drop= True)

In [11]:
unique['buzday'] = unique['buzday'] - 1
unique['age2'] = np.where(unique['Last_Call'].isna(), unique.age, unique.buzday)
f1 = unique.sla >= unique.age2
unique['meet_sla'] = np.where(f1, 1,0)
f1 = unique['Last_Call'] == '2022-02-22'
f2 = unique.Project_Type == 'UHC HEDIS'
unique[f1 & f2][['Last_Call','age','buzday','age2']]

,Last_Call,age,buzday,age2
2670,2022-02-22,11.0,8.0,8.0
2692,2022-02-22,11.0,8.0,8.0
2693,2022-02-22,11.0,8.0,8.0
2694,2022-02-22,11.0,8.0,8.0
2742,2022-02-22,11.0,8.0,8.0
...,...,...,...,...
125354,2022-02-22,8.0,8.0,8.0
125526,2022-02-22,8.0,8.0,8.0
125585,2022-02-22,8.0,8.0,8.0
125613,2022-02-22,8.0,8.0,8.0


In [12]:
def match(df, match):
    group = 'Project_Type'
    # full df, group by location & project, get smallest value for the match in each projects
    t2 = df.groupby([match, group]).agg({'sla':'min','days_before_out_sla':'min'}).reset_index()
    # group projects by location | match
    t2['groups'] = t2.groupby(match)[group].transform(lambda x: ' | '.join(x))
    # group by group of projects, find smallest value for each location
    t3 = t2.groupby(['groups',match]).agg({'sla':'min','days_before_out_sla':'min'}).reset_index()
    # pivot groups by smallest value, counting sites
    gp = t3.groupby(['groups','sla','days_before_out_sla'])[match].count()
    return gp 

def buckets(df):
    split = {
        'RADV':'RADV',
        'Medicare': '(Chart Sync|HCR)',
        'Medicaid':'(Chart Review|Medicaid- HospCR|Clinical Review MCaid PhyCR)',
        'ACA':'(ACA-PhysicianCR|ACA-HospitalCR)',
        'UHC HEDIS':'UHC HEDIS',
        'HEDIS':'HEDIS_c'
        }
    # split = {
    #     'Medicare Risk': 'Medicare Risk',
    #     'Medicaid Risk':'Medicaid Risk',
    #     'ACA':'ACA',
    #     'RADV':'RADV',
    #     'HEDIS':'HEDIS'
    #     }

    df['Bucket'] = 'none'

    for bucket, search in split.items():
        f1 = df.groups.apply(lambda x: bool(re.search(search, x)))
        df.Bucket = np.where(f1, bucket, df.Bucket)
    return df

def sites(df0, projects, project):
    # create column
    df0['days_before_out_sla'] = df0.sla - df0.age2 + 1
    df0['days_before_out_sla'] = np.where(df0.days_before_out_sla < 0, 0, df0.days_before_out_sla)
    
    # f1 = df0.Project_Type.isin([x for x in projects if x not in project])
    # df0['days_before_out_sla'] = np.where(f1, 11, df0.days_before_out_sla)
    
    # change HEDIS Commercail
    df = df0[df0.Project_Type.isin(projects)].copy()
    
    f1 = df.Project_Type == 'HEDIS'
    df.Project_Type = np.where(f1, 'HEDIS_c', df.Project_Type)

    cf = df[(df.MasterSiteId == 1000838) | (df.MasterSiteId.isna())]
    msid = df[(df.MasterSiteId != 1000838) & (df.MasterSiteId.notna())]

    t  = match(cf,'PhoneNumber')
    t2 = match(msid,'MasterSiteId')
    t3 = (t2 + t).reset_index()
    t3.columns = ['groups','sla','days_before_out_sla','count']
    return buckets(t3)


In [13]:
uhc = ['UHC HEDIS']
hedis = ['HEDIS']
aca = ['ACA-PhysicianCR','ACA-HospitalCR'] #, 'RADV'
medicaid = ['Chart Review','Medicaid- HospCR', 'Clinical Review MCaid PhyCR']
radv = ['RADV']
projects = [uhc, hedis, aca, medicaid, radv]

def msls(ls):
    l = []
    for i in ls:
        if i is list():
            l += msls(i)
        else:
            l += i
    return l

In [14]:
tt = sites(unique, msls(projects), radv)
piv = tt.pivot_table(index='days_before_out_sla', columns='Bucket', values='count', aggfunc='sum',margins=True)
piv.to_clipboard()
# piv
piv

Bucket,ACA,HEDIS,Medicaid,RADV,UHC HEDIS,All
days_before_out_sla,,,,,,
0.0,15498.0,4404.0,5776.0,448.0,5335.0,31461.0
1.0,966.0,1526.0,771.0,7.0,1432.0,4702.0
2.0,1490.0,384.0,1363.0,10.0,2995.0,6242.0
3.0,1948.0,340.0,858.0,14.0,4237.0,7397.0
4.0,2171.0,799.0,900.0,0.0,4363.0,8233.0
5.0,2966.0,1995.0,1019.0,0.0,2189.0,8169.0
6.0,2766.0,NaN,NaN,NaN,NaN,2766.0
7.0,1970.0,NaN,NaN,NaN,NaN,1970.0
8.0,1269.0,NaN,NaN,NaN,NaN,1269.0
